# Notebook 3: Generate draft EcN model

This notebook is based on the work of Norsigian _et al._ (2019). This notebook utilizes the output of notebook 2 (presence/absence matrix and geneID matrix) to generate a draft model of _E. coli_ Nissle from 55 reference models. This is done in the following steps: 

1. Deletion of missing genes/reaction from reference model iML1515 to generate draft model
2. Update gene-protein-reaction rule
3. Comparison with 54 reference models based on orthology
4. Addition of reactions originating from genes encoding for proteins with >80% homology
5. Update gene-protein-reaction rule after each model comparison

In [1]:
#import package needed
import cobra
import numpy as np
import pandas as pd
from glob import glob
from cobra.io import load_json_model
from cobra import Model, Reaction, Metabolite
from cobra.manipulation.delete import delete_model_genes, remove_genes
from cobra.manipulation.modify import rename_genes

# 1. Creation of EcN draft model from iML1515

###  1.1 Deletion of missing genes/reaction from reference model 

In [2]:
#Load the base E. coli reconstruction iML1515
model = load_json_model('../data/models/iML1515.json')
model

Name,iML1515
Memory address,0x026d476de048
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [3]:
## Load the previously generated homology matrix for MG1655 vs. EcN
hom_matrix=pd.read_csv('../tables/MG_ortho_matrix.csv')
hom_matrix=hom_matrix.set_index('Unnamed: 0')

# Give the ID of the assembly used
EcN_ID = "CP022686.1"

In [4]:
hom_matrix.head()

,CP022686.1
Unnamed: 0,
b2551,1.0
b0870,1.0
b3368,1.0
b2436,1.0
b3500,1.0


### 1.2 Delete missing genes from a copy of the iML1515 model
Start with the iML1515 model, identify the missing genes from the matrix, and remove them and their associated reactions from the reference model

In [5]:
#Get the list of Gene IDs from the homology matrix dataframe for the current strain without a homolog
currentStrain=hom_matrix[EcN_ID]
nonHomologous=currentStrain[currentStrain==0.0]
nonHomologous=nonHomologous.index.tolist()
    
#s0001 is an artificial gene used in iML1515 for spontaneous reactions and as such has no homologs,
#However, it is retained for these spontaneous reactions to function
nonHomologous.remove('s0001')
    
#Define a list of Gene objects from the base reconstruction to be deleted from the current strain
toDelete=[]
for gene in nonHomologous:
    toDelete.append(model.genes.get_by_id(gene))

#Establish a model copy and remove the reactions that should not be present according to bbh in notebook 2
modelCopy=model.copy()
remove_genes(modelCopy, toDelete, remove_reactions=True)
modelCopy.id= EcN_ID

# Save reduced iML1515 model as EcN model
cobra.io.json.save_json_model(modelCopy, str('../data/models/%s_MG.json'%EcN_ID), pretty=False)

### 1.3 Record gene/reaction addition process 

In [6]:
# Record which genes and reactions are present at this stage of the model building process
rxn_origin_df = pd.DataFrame(columns=['reaction', 'origin', 'added', 'notebook', 'notes'])
rxn_origin_df.set_index('reaction', inplace=True)

gene_origin_df = pd.DataFrame(columns=['gene', 'EcN_gene', 'origin', 'added', 'notebook', 'notes'])
gene_origin_df.set_index('gene', inplace=True)

for gene in modelCopy.genes:
    gene_origin_df.loc[gene.id, 'EcN_gene'] = gene.id
    gene_origin_df.loc[gene.id, 'origin'] = 'iML1515'
    gene_origin_df.loc[gene.id, 'added'] = 'automatic'
    gene_origin_df.loc[gene.id, 'notebook'] = '3.1'
    gene_origin_df.loc[gene.id, 'notes'] = 'none'
        
for rxn in modelCopy.reactions:
    rxn_origin_df.loc[rxn.id, 'origin'] = 'iML1515'
    rxn_origin_df.loc[rxn.id, 'added'] = 'automatic'
    rxn_origin_df.loc[rxn.id, 'notebook'] = '3.1'
    rxn_origin_df.loc[rxn.id, 'notes'] = 'none'

### 1.4 Update Model Gene Product Rules

In [7]:
# Get the geneIDs, set index and remove all genes == None
MG_geneIDs_matrix = pd.read_csv('../tables/MG_geneIDs_matrix.csv')
MG_geneIDs_matrix.set_index('Unnamed: 0', inplace=True)
MG_geneIDs_matrix = MG_geneIDs_matrix[MG_geneIDs_matrix.index != 'None']

MG_geneIDs_matrix.head()

,CP022686.1
Unnamed: 0,
b2551,CIW80_06560
b0870,CIW80_22360
b3368,CIW80_11380
b2436,CIW80_06055
b3500,CIW80_12105


In [8]:
# Utilize the geneIDs matrix to update the GPRs in the EcN model with the proper gene ID
mod = '../data/models/%s_MG.json'%EcN_ID
model=cobra.io.load_json_model(mod)

IDMapping = MG_geneIDs_matrix[EcN_ID].to_dict()
IDMappingParsed = {k:v for k,v in IDMapping.items() if v != 'None'}
rename_genes(model,IDMappingParsed)

# Save the model
cobra.io.json.save_json_model(model,mod, pretty=False)

In [9]:
rxn_origin_df

,origin,added,notebook,notes
reaction,,,,
ALATA_D2,iML1515,automatic,3.1,none
SHCHD2,iML1515,automatic,3.1,none
CPPPGO,iML1515,automatic,3.1,none
GTHOr,iML1515,automatic,3.1,none
DHORD5,iML1515,automatic,3.1,none
...,...,...,...,...
SUCCt1pp,iML1515,automatic,3.1,none
QUINDH,iML1515,automatic,3.1,none
LCARSyi,iML1515,automatic,3.1,none


### 1.5 Record new gene names

In [10]:
# Rename the gene names to EcN IDs in the origin table
gene_origin_df['EcN_gene'] = gene_origin_df['EcN_gene'].map(IDMapping)

In [11]:
# Save as a table
rxn_origin_df.to_csv('../tables/rxn_origin.csv')
gene_origin_df.to_csv('../tables/gene_origin.csv')

### 1.6 Examine the draft strain specific model contents

In [12]:
# Gather the general information on the draft models
print (modelCopy.id,'Number of Model Genes:',len(modelCopy.genes),'Number of Model Reactions:',len(model.reactions))

CP022686.1 Number of Model Genes: 1390 Number of Model Reactions: 2607


# 2. Gene addition to draft EcN model
Per strain:
1. Identify IDs of all gene homologies <ortho == 1>
2. Create a list of all orthologous genes (def ortho_to_list)
    
Within “strain loop”: Per reaction in ortho list
1. Relate gene ID to reaction in strain’s model
2. Check presence of the reaction in EcN draft model
3. If not present, check homology of all genes related to reaction
4. If all genes homologous, add reaction to draft model
    
Within “strain loop”: After reactions are added:
1. Update the GPRs to Nissle gene IDs
2. Add the genes/reactions to the overview table

### 2.1 OrthoIDs_matrix creation

In [13]:
# Get the number of genes represented within the 55 models
geneIDs_matrix = pd.read_csv('../tables/geneIDs_matrix.csv')
geneIDs_matrix.set_index('Unnamed: 0', inplace=True)

# Combine with ortho_matrix and remove all genes that do not have homology above 80%
ortho_matrix = pd.read_csv('../tables/ortho_matrix.csv')
ortho_matrix.set_index('Unnamed: 0', inplace=True)

for strain in ortho_matrix.columns:
    geneIDs_matrix[strain] = np.where(ortho_matrix[strain] == 0, 'None', geneIDs_matrix[strain])

geneIDs_matrix.head()

,CU651637,CP001855,CP002167,CP000468,CP000946,CP000819,CP001665,AM946981,CP001509,CP001396,...,CU928163,CP000243,CP001063,CP000036,CP000034,CP001383,AE014073,AE005674,CP000266,CP000038
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CIW80_00005,LF82_1032,NRG857_06995,UM146_09940,APECO1_564,EcolC_2245,ECB_01368,ECBD_2227,B21_01380,ECD_01368,BWG_1240,...,ECUMN_1659,UTI89_C1635,SbBS512_E1633,SBO_1674,SDY_1774,SFxv_2017,S1472,SF1802,SFV_1794,SSON_1733
CIW80_00010,LF82_2792,NRG857_07000,UM146_09935,APECO1_565,EcolC_2244,ECB_01369,ECBD_2226,B21_01381,ECD_01369,BWG_1241,...,ECUMN_1662,UTI89_C1636,SbBS512_E1634,SBO_1673,None,SFxv_2015,None,None,SFV_1792,SSON_1732
CIW80_00015,LF82_0064,NRG857_07005,UM146_09930,APECO1_566,EcolC_2243,ECB_01370,ECBD_2225,B21_01382,ECD_01370,BWG_1242,...,ECUMN_1663,UTI89_C1637,SbBS512_E1635,SBO_1672,None,SFxv_2014,S1475,SF1797,SFV_1791,None
CIW80_00020,LF82_0802,NRG857_07010,UM146_09925,APECO1_567,EcolC_2242,ECB_01371,ECBD_2224,B21_01383,ECD_01371,None,...,ECUMN_1664,UTI89_C1638,SbBS512_E1636,SBO_1671,None,None,None,SF1795,SFV_1790,SSON_1725
CIW80_00025,LF82_0397,NRG857_07015,UM146_09920,APECO1_568,EcolC_2241,ECB_01374,ECBD_2221,B21_01386,ECD_01374,BWG_1243,...,ECUMN_1665,UTI89_C1639,SbBS512_E1642,SBO_1669,SDY_1764,SFxv_2012,S1478,SF1794,SFV_1789,SSON_1724


In [14]:
# save as orthoIDs_matrix
orthoIDs_matrix = geneIDs_matrix
orthoIDs_matrix.to_csv('../tables/orthoIDs_matrix.csv')

## 2.2 Gene addition

In [15]:
# From the ortho_matrix, create a list of genes for the CurrentStrain that have homology with EcN genes 
def ortho_to_list(CurrentStrain, geneIDs_matrix):

    # Load orthology matrix and drop all similarities with existing EcN model
    ortho_matrix = pd.read_csv('../tables/ortho_matrix.csv')
    ortho_matrix.set_index('Unnamed: 0', inplace=True)

    # Rename the current strain in the geneIDs_matrix
    current_geneIDs_matrix = geneIDs_matrix[CurrentStrain].rename('geneID')

    # Merge ortho_matrix with current geneIDs matrix
    ortho_matrix = pd.merge(ortho_matrix, current_geneIDs_matrix, left_index=True, right_index=True)
    ortho_matrix.set_index('geneID', inplace=True)
    try: ortho_matrix.drop('None', inplace=True)
    except: pass

    # Create list of all genes that are not yet in model and have homology with current strain
    ortho_list = ortho_matrix[ortho_matrix[CurrentStrain] == 1.0].index.tolist()
    return(ortho_list)

In [16]:
# Based on the gene IDs in the ortho_list, retrieve reactions from the current_model and add them to the EcN model
def gene_addition(modelCopy, current_model, ortho_list):
    add_gene = []
    
    # Record which genes and reactions are added and from which model at this stage of the model building process
    g_origin_df = pd.DataFrame(columns=['gene', 'EcN_gene', 'origin', 'added', 'notebook', 'notes'])
    g_origin_df.set_index('gene', inplace=True)
    
    r_origin_df = pd.DataFrame(columns=['reaction', 'origin', 'added', 'notebook', 'notes'])
    r_origin_df.set_index('reaction', inplace=True)
    
    # for all genes that have homology, check if they should be added
    for gene in ortho_list:
        try:
            reactions = current_model.genes.get_by_id(gene).reactions
            for reaction in reactions:
                
                # check if the gene is already present in the model
                try:
                    modelCopy.reactions.get_by_id(reaction.id)
                
                # If the reaction is not yet in the model, check the dependencies of genes linked to the reaction
                # If the model contains an "and" statement, check if all involved genes are present in the ortho_list
                # If they are not all present, do not add the reaction
                except: 
                    grr = current_model.reactions.get_by_id(reaction.id).gene_reaction_rule # get gene reaction rule
                    or_statement = grr.split(' or ') #split based on "or" statement
                    for or_genes in or_statement:
                        if len(or_genes.split(' and ')) == 1: #check whether there is an "and" statement
                            add_gene.append(True) #if not there are only "or" statements, and having one of them in the ortho_list is sufficient
                        else:
                            and_present = True
                            involved_genes = or_genes.split(' and ') #create a list of all the genes in the "and" statement
                            for gene_x in involved_genes:
                                if gene_x.lstrip('(').rstrip(')') not in ortho_list: #check for each gene if it's present in the ortho_list
                                    and_present = False #if one of the genes is not present, do not give True for addition on this part
                            if and_present == True:
                                add_gene.append(True)
                                
                    if True in add_gene: #only if one of the "or" statements (one gene or combination of "and" genes) is completely present, add reaction
                        modelCopy.add_reactions([current_model.reactions.get_by_id(reaction.id)]) # if not present, add
                        print('added:', reaction.id)
                        
                        # Add gene & reaction to overview table
                        g_origin_df.loc[gene, 'EcN_gene'] = gene
                        g_origin_df.loc[gene, 'origin'] = current_model
                        g_origin_df.loc[gene, 'added'] = 'automatic'
                        g_origin_df.loc[gene, 'notebook'] = '3.1'
                        g_origin_df.loc[gene, 'notes'] = 'none'
                        
                        r_origin_df.loc[reaction.id, 'origin'] = current_model
                        r_origin_df.loc[reaction.id, 'added'] = 'automatic'
                        r_origin_df.loc[reaction.id, 'notebook'] = '3.1'
                        r_origin_df.loc[reaction.id, 'notes'] = 'none'
                        
                        # Reset the "add_gene" list
                        add_gene = []
                    else:
#                         print('Not added:', reaction.id)
                        continue
        except:
            continue
    return(modelCopy, g_origin_df, r_origin_df)

In [17]:
# Load the MG1655 model from which all reactions not present in EcN were removed
EcN_model = cobra.io.load_json_model('../data/models/%s_MG.json'%EcN_ID)
modelCopy = EcN_model.copy()

# Load the table with information on origin of genes & reactions
gene_origin_df = pd.read_csv('../tables/gene_origin.csv')
gene_origin_df.set_index('gene', inplace=True)

rxn_origin_df = pd.read_csv('../tables/rxn_origin.csv')
rxn_origin_df.set_index('reaction', inplace=True)

# Load strain info to get the model for each strain and remove MG1655, so it will not be used in the comparison
strain_info = pd.read_csv('../tables/strain_info.csv', usecols=[1,3,6])
strain_info = strain_info[strain_info['NCBI ID'] != 'U00096']

#load the orthoID matrix. Set index, rename EcN and remove all genes == None
orthoIDs_matrix = pd.read_csv('../tables/orthoIDs_matrix.csv')
orthoIDs_matrix = orthoIDs_matrix.set_index('Unnamed: 0') #Set EcN gene IDs as index
orthoIDs_matrix = orthoIDs_matrix[orthoIDs_matrix.index != 'None'] # Remove all indexes 00 'None'

orthoIDs_matrix.head()

,CU651637,CP001855,CP002167,CP000468,CP000946,CP000819,CP001665,AM946981,CP001509,CP001396,...,CU928163,CP000243,CP001063,CP000036,CP000034,CP001383,AE014073,AE005674,CP000266,CP000038
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
CIW80_00005,LF82_1032,NRG857_06995,UM146_09940,APECO1_564,EcolC_2245,ECB_01368,ECBD_2227,B21_01380,ECD_01368,BWG_1240,...,ECUMN_1659,UTI89_C1635,SbBS512_E1633,SBO_1674,SDY_1774,SFxv_2017,S1472,SF1802,SFV_1794,SSON_1733
CIW80_00010,LF82_2792,NRG857_07000,UM146_09935,APECO1_565,EcolC_2244,ECB_01369,ECBD_2226,B21_01381,ECD_01369,BWG_1241,...,ECUMN_1662,UTI89_C1636,SbBS512_E1634,SBO_1673,None,SFxv_2015,None,None,SFV_1792,SSON_1732
CIW80_00015,LF82_0064,NRG857_07005,UM146_09930,APECO1_566,EcolC_2243,ECB_01370,ECBD_2225,B21_01382,ECD_01370,BWG_1242,...,ECUMN_1663,UTI89_C1637,SbBS512_E1635,SBO_1672,None,SFxv_2014,S1475,SF1797,SFV_1791,None
CIW80_00020,LF82_0802,NRG857_07010,UM146_09925,APECO1_567,EcolC_2242,ECB_01371,ECBD_2224,B21_01383,ECD_01371,None,...,ECUMN_1664,UTI89_C1638,SbBS512_E1636,SBO_1671,None,None,None,SF1795,SFV_1790,SSON_1725
CIW80_00025,LF82_0397,NRG857_07015,UM146_09920,APECO1_568,EcolC_2241,ECB_01374,ECBD_2221,B21_01386,ECD_01374,BWG_1243,...,ECUMN_1665,UTI89_C1639,SbBS512_E1642,SBO_1669,SDY_1764,SFxv_2012,S1478,SF1794,SFV_1789,SSON_1724


In [18]:
# Loop over the strains and add reactions
for strain in strain_info['NCBI ID']:
    CurrentStrain = strain

    # Load model
    CurrentModel = strain_info.loc[strain_info['NCBI ID'] == CurrentStrain, 'Model Name'].values[0]
    current_model = cobra.io.load_json_model('../data/models/%s.json'%CurrentModel)
    print("Adding reactions from strain: ", CurrentStrain,', model:', current_model.id)
    
    # Get ortho_list without reactions already present in model
    ortho_list = ortho_to_list(CurrentStrain, orthoIDs_matrix)
    
    # Add reactions of each model to the EcN model
    modelCopy, g_origin_df, r_origin_df = gene_addition(modelCopy, current_model, ortho_list)
    
    #Utilize the geneIDs matrix to update the GPRs in each of the strain-specific models with the proper gene ID
    IDMapping=orthoIDs_matrix[CurrentStrain].to_dict()
    IDMappingParsed = {v:k for k,v in IDMapping.items() if v != 'None'}  # v:k to make the CurrentStrain geneIDs keys and EcN gene IDs values
    rename_genes(modelCopy,IDMappingParsed)
    
    # Change the gene ID to EcN gene ID in overview table
    IDMapping_switch = {y:x for x,y in IDMapping.items()}
    g_origin_df['EcN_gene'] = g_origin_df['EcN_gene'].map(IDMapping_switch)
    gene_origin_df = gene_origin_df.append(g_origin_df)
                                           
    rxn_origin_df = rxn_origin_df.append(r_origin_df)
    
    # Model evaluation
    print ('Number of Model Genes:',len(modelCopy.genes),'Number of Model Reactions:',len(modelCopy.reactions), '\n')

Adding reactions from strain:  CU651637 , model: iLF82_1304
added: DARBt3ipp
added: CLB6PH
added: CLBH
added: SALCNH
added: CLBtpp
added: SALCtpp
added: CLBptspp
added: SALCptspp
added: CELLBpts_1
added: TYRTRS
added: GTHPi_copy2
added: GTHPi_copy1
added: PHETRS
added: THRTRS
added: TAG1PK
added: DC6PH
added: CHITPH
added: CHITOBpts
added: ASPTRS
added: ARGTRS
added: DARBabcpp
added: SALSYN
added: FEpp
added: FEex
added: ADOCBLPS
added: METTRS
added: SALCHS4FEtonex
added: DMALRED
added: PACt1
added: GULNLtex
added: 3PGtex
added: TAGtex
added: RAFFtex
added: 4HTHRtex
added: PENGt1
added: 3hoxpactex
added: HPAtex
added: DRIBtex
added: TETtex
added: DARBtex
added: DXYLUDtex
added: ICITtex
added: 2PGtex
added: OAAtex
added: GALAMtex
added: AEPtex
added: AIRStex
added: SALCNtex
added: FOAMtex
added: 6APAt1
added: ACONCtex
added: SALCHSXtex
added: 4HOXPACt2pp
added: UPLA4NT
added: NPHS
added: HACD7i
added: HACD5i
added: HACD1i
added: HACD8i
added: HACD6i
added: DHACOAH
added: HACD3i
added: H

Adding reactions from strain:  CP000800 , model: iEcE24377_1341
Number of Model Genes: 1476 Number of Model Reactions: 2859 

Adding reactions from strain:  FN649414 , model: iETEC_1333
Number of Model Genes: 1476 Number of Model Reactions: 2859 

Adding reactions from strain:  CP002729 , model: iUMNK88_1353
Number of Model Genes: 1476 Number of Model Reactions: 2859 

Adding reactions from strain:  CP001969 , model: iECOK1_1307
Number of Model Genes: 1476 Number of Model Reactions: 2859 

Adding reactions from strain:  CU928161 , model: iECS88_1305
added: CELLBpts
Number of Model Genes: 1476 Number of Model Reactions: 2860 

Adding reactions from strain:  CP000247 , model: iECP_1309
Number of Model Genes: 1476 Number of Model Reactions: 2860 

Adding reactions from strain:  CP001671 , model: iECABU_c1320
Number of Model Genes: 1476 Number of Model Reactions: 2860 

Adding reactions from strain:  AE014075 , model: ic_1306
Number of Model Genes: 1476 Number of Model Reactions: 2860 

Ad

### 2.3 Inspection of draft model & reaction/gene tables

In [19]:
# Final model evaluation
print ('Number of Model Genes:',len(modelCopy.genes),'Number of Model Reactions:',len(modelCopy.reactions))

# Check the correct length of the rxn_origin_dataframe
print('The rxn_origin dataframe has the correct length:', len(modelCopy.reactions) == len(rxn_origin_df))

Number of Model Genes: 1478 Number of Model Reactions: 2867
The rxn_origin dataframe has the correct length: True


In [20]:
rxn_origin_df.head()

,origin,added,notebook,notes
reaction,,,,
ALATA_D2,iML1515,automatic,3.1,none
SHCHD2,iML1515,automatic,3.1,none
CPPPGO,iML1515,automatic,3.1,none
GTHOr,iML1515,automatic,3.1,none
DHORD5,iML1515,automatic,3.1,none


In [21]:
gene_origin_df.head()

,EcN_gene,origin,added,notebook,notes
gene,,,,,
b2551,CIW80_06560,iML1515,automatic,3.1,none
b0870,CIW80_22360,iML1515,automatic,3.1,none
b3368,CIW80_11380,iML1515,automatic,3.1,none
b2436,CIW80_06055,iML1515,automatic,3.1,none
b3500,CIW80_12105,iML1515,automatic,3.1,none


### 2.4 Save model & reaction/gene tables

In [22]:
# Save as a table
rxn_origin_df.to_csv('../tables/rxn_origin.csv')
gene_origin_df.to_csv('../tables/gene_origin.csv')

In [23]:
# Save the model
cobra.io.json.save_json_model(modelCopy, str('../data/models/%s_draft.json'%EcN_ID), pretty=False)